In [12]:
from tensorflow.compat.v1.keras.datasets import mnist
from tensorflow.compat.v1.keras.models import Sequential, Model
from tensorflow.compat.v1.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation
from tensorflow.compat.v1.keras.utils import to_categorical
# from tensorflow.python.keras.backend import *
from tensorflow.compat.v1.keras import backend as K
import tensorflow.compat.v1 as tf
import numpy as np
tf.disable_v2_behavior()

# from tensorflow.python.keras.datasets import mnist
# from tensorflow.python.keras.models import Sequential, Model
# from tensorflow.python.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation
# from tensorflow.python.keras.utils import to_categorical
# # from tensorflow.python.keras.backend import *
# from tensorflow.python.keras import backend as K
# import tensorflow as tf
# import numpy as np
# # import tensorflow.compat.v1 as tf
# # tf.disable_v2_behavior()


trgt_inp = tf.placeholder(tf.float32, (10, 32, 32, 1))

Instructions for updating:
non-resource variables are not supported in the long term


In [13]:
# import tf.keras.backend as K
import math

(x_train, y_train), (x_test, y_test) = mnist.load_data()

batch_size = 32
epochs = 2

n_samples = x_train.shape[0]

img_width = x_train.shape[1]
img_height = x_train.shape[2]
channels = 1

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

n_classes = y_train.shape[1]

print('Train size - {}. Each img {}x{} pixels.'.format(x_train.shape[0], img_width, img_height))

x_train = x_train.reshape((x_train.shape[0], img_width, img_height, 1))
x_test = x_test.reshape((x_test.shape[0], img_width, img_height, 1))

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255

Train size - 60000. Each img 28x28 pixels.


In [15]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(img_width, img_height, channels)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(n_classes))

sess = K.get_session()  # Get the current session from Keras.
def loss_fn(correct, predicted):
    return tf.nn.softmax_cross_entropy_with_logits(labels=correct,
                                                   logits=predicted)


model.compile(loss=loss_fn, optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_5 (Dropout)          (None, 128)              

In [16]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 10s 159us/sample - loss: 0.1858 - acc: 0.9444 - val_loss: 0.0435 - val_acc: 0.9853
Epoch 2/2
60000/60000 [==============================] - 9s 154us/sample - loss: 0.0800 - acc: 0.9759 - val_loss: 0.0406 - val_acc: 0.9871


In [19]:
"""FGSM ATTACK"""

n_adversarial = y_test.shape[0]

trgt_inp = tf.placeholder(tf.float32, (n_adversarial, img_width, img_height, 1))
true_out = tf.placeholder(tf.float32, (n_adversarial, 10))
epsilon = tf.placeholder(tf.float32, [n_adversarial, img_width, img_height, 1])

new_inp = tf.identity(trgt_inp)  #

output = model(new_inp)

loss = tf.sqrt(tf.nn.softmax_cross_entropy_with_logits(logits=output, labels=true_out))

# Constructs symbolic partial derivatives of sum of ys w.r.t. x in xs.
grad = tf.gradients(loss, new_inp)
# will allow gradient computation only on the input (#check this)
new_inp = tf.stop_gradient(new_inp + epsilon * tf.sign(grad))

step = 0.9
eps = np.full((n_adversarial, img_width, img_height, 1), step)

feed_dict = {trgt_inp: x_test,
             true_out: y_test,
             epsilon: eps}

x_adv, l = sess.run([new_inp, loss],
                    feed_dict)

x_adv = x_adv.reshape((n_adversarial, img_width, img_height, 1))

acc_bfr_attck = model.evaluate(x=x_test, y=y_test, batch_size=32)[1]
acc_aftr_attck = model.evaluate(x=x_adv, y=y_test, batch_size=32)[1]
print('Accuracy before FGSM {}, Accuracy after FGSM {}'.format(acc_bfr_attck, acc_aftr_attck))

10000/10000 [==============================] - 1s 70us/sample - loss: 14.2489 - acc: 0.1574
Accuracy before FGSM 0.9871000051498413, Accuracy after FGSM 0.1573999971151352
